In [1]:
import os
import sys
sys.path.insert(0, os.path.abspath('../'))
from service.Classifier.DataLoader.DataLoader import DataLoader
from service.Classifier.Enums.priority import Priority
from service.Classifier.DataLoader.P2DataLoader import P2DataLoader
from service.Classifier.DataLoader.P3DataLoader import P3DataLoader
from service.Classifier.DataLoader.P4DataLoader import P4DataLoader
from service.Classifier.Model.XGBRegressionModel import XGBRegressionModel
from service.Classifier.PreProcessing.RegressionModelPreProcessor import RegressionModelPreProcessor
from service.Classifier.DataSplit.TrainTestSplit import TrainTestSplit
from service.Classifier.Enums.dataTypeEnum import DataType

/Users/fjirigesi/Documents/OOSLAClassifier/service/Classifier/Model/XGBRegressionModel.py:2: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  from pandas import np


In [2]:
def initializeByPriority(priority):
    # load new updated data via Features class
    if priority == Priority.P2.value:
        return P2DataLoader()
    elif priority == Priority.P3.value:
        return P3DataLoader()
    elif priority == Priority.P4.value:
        return P4DataLoader()
    return None

In [3]:
priority = Priority.P2.value
def extract_data(priority):
    dataLoader = initializeByPriority(priority)
    data_df = dataLoader.loadTrainingRawData()
    # split data into train test 4:1
    dataSplit = TrainTestSplit()
    train_df, test_df = dataSplit.split(data_df)

    # generate features
    train_df, feature_names = dataLoader.transformRawDataToFeatures(train_df, DataType.TRAINDATA.value)
    test_df, feature_names = dataLoader.transformRawDataToFeatures(test_df, DataType.VALIDATION.value)
    # pre-processing the data based on model type
    preprocessor = RegressionModelPreProcessor(feature_names)
    train_X, train_y, test_X, test_y = preprocessor.preprocessing(train_df, test_df)
    
    return (train_X, train_y, test_X, test_y, train_df, test_df)

In [4]:
import xgboost as xgb
import numpy as np
import pandas as pd
from service.Classifier.Enums.LabelEnum import LabelEnum
from sklearn.metrics import mean_squared_error, mean_absolute_error, confusion_matrix, classification_report
from service.Classifier.PreProcessing.Utils.dataValidator import DataValidator

def evaluate_result(train_X, train_y, test_X, test_y, train_df, test_df):

    model = xgb.XGBRegressor(max_depth=5, n_estimators=100, learning_rate=0.05).fit(train_X, train_y)
    predictions = np.rint(model.predict(test_X)).astype(np.int64)
    submission = pd.DataFrame({'ID': test_df['keyID'],
                               LabelEnum.CLOSEDDAY.value: predictions, 'Actual': test_y})
    rmse = np.sqrt(mean_squared_error(predictions, test_df[LabelEnum.CLOSEDDAY.value]))
    print("RMSE: %f" % rmse)
    mae = mean_absolute_error(predictions, test_y)
    print("MAE: %f" % mae)
    DataValidator().valid(submission, priority, test_df)
    print(confusion_matrix(submission["truth"], submission["prediction"]))
    print(classification_report(submission["truth"], submission["prediction"]))
    
    return (test_y, predictions, submission["truth"], submission["prediction"])

In [5]:
priority = Priority.P2.value

train_X, train_y, test_X, test_y, train_df, test_df = extract_data(priority)

p2_regression_true, p2_regression_pred, p2_classification_true, p2_classification_pred = evaluate_result(train_X, train_y, test_X, test_y, train_df, test_df)

/Users/fjirigesi/anaconda3/lib/python3.8/site-packages/pymongo/common.py:781: UserWarning: The value of ssl must be 'true' or 'false'
  warnings.warn(str(exc))


The project is not in embedding: NATBEMCR
The project is not in embedding: PX
The project is not in embedding: GGRIP
The project is not in embedding: PROBLEM
The project is not in embedding: CSCNPLAT
RMSE: 9.690657
MAE: 7.766382
[[211 185]
 [ 89 217]]
              precision    recall  f1-score   support

       False       0.70      0.53      0.61       396
        True       0.54      0.71      0.61       306

    accuracy                           0.61       702
   macro avg       0.62      0.62      0.61       702
weighted avg       0.63      0.61      0.61       702



In [6]:
priority = Priority.P3.value

train_X, train_y, test_X, test_y, train_df, test_df = extract_data(priority)

p3_regression_true, p3_regression_pred, p3_classification_true, p3_classification_pred = evaluate_result(train_X, train_y, test_X, test_y, train_df, test_df)

/Users/fjirigesi/anaconda3/lib/python3.8/site-packages/pymongo/common.py:781: UserWarning: The value of ssl must be 'true' or 'false'
  warnings.warn(str(exc))


The project is not in embedding: RISKVRTB
The project is not in embedding: MPPMO
The project is not in embedding: NRP
The project is not in embedding: WARROOM
The project is not in embedding: PROBLEM
The project is not in embedding: WEBRES
RMSE: 20.065636
MAE: 14.088812
[[520  87]
 [145 115]]
              precision    recall  f1-score   support

       False       0.78      0.86      0.82       607
        True       0.57      0.44      0.50       260

    accuracy                           0.73       867
   macro avg       0.68      0.65      0.66       867
weighted avg       0.72      0.73      0.72       867



In [7]:
priority = Priority.P4.value

train_X, train_y, test_X, test_y, train_df, test_df = extract_data(priority)

p4_regression_true, p4_regression_pred, p4_classification_true, p4_classification_pred = evaluate_result(train_X, train_y, test_X, test_y, train_df, test_df)

/Users/fjirigesi/anaconda3/lib/python3.8/site-packages/pymongo/common.py:781: UserWarning: The value of ssl must be 'true' or 'false'
  warnings.warn(str(exc))


The project is not in embedding: SDOCATOP
RMSE: 24.272358
MAE: 16.646316
[[320  27]
 [ 83  45]]
              precision    recall  f1-score   support

       False       0.79      0.92      0.85       347
        True       0.62      0.35      0.45       128

    accuracy                           0.77       475
   macro avg       0.71      0.64      0.65       475
weighted avg       0.75      0.77      0.74       475



In [8]:
train_df

,keyID,priority,projectCoverage,ResolvedDay,ClosedDay
0,COMS-8231,P4,0.004162,65,65
1,CANCEL-6080,P4,0.004131,68,68
2,BILLING-16551,P4,0.038925,49,49
3,CANCEL-6067,P4,0.004836,78,78
4,BILLING-16534,P4,0.040816,56,56
...,...,...,...,...,...
1910,VOD-3573,P4,0.056915,68,68
1911,CANCEL-6268,P4,0.004724,80,80
1912,RES-20850,P4,0.008016,86,86
1913,BILLING-17524,P4,0.045840,86,86
